<a href="https://colab.research.google.com/github/rafamayo/MTEK/blob/main/Biosignals_2_Digital_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Digital Filtering - A Fundamental Pre-Processing Step

Based on: http://notebooks.pluxbiosignals.com/notebooks/Categories/Pre-Process/digital_filtering_rev.html

The electrophysiological signals acquired during measurement contain two intrinsic components: the signal of interest and noise.

The signal is the data we intend to study, while noise represents any unwanted interference or variability that is not relevant to the analysis.

Noise can originate from various sources, including random events or voluntary/involuntary movements by the subject that disrupt the <a href="https://en.wikipedia.org/wiki/Noise_(signal_processing)" target="_blank">signal acquisition</a> process.

To maximize the <a href="https://en.wikipedia.org/wiki/Signal-to-noise_ratio" target="_blank">signal-to-noise ratio</a>, <a href="https://en.wikipedia.org/wiki/Filter_(signal_processing)" target="_blank">filtering</a> is a fundamental step. Filtering can be implemented through hardware (using analog filters) or software (using digital filters), each playing a critical role in reducing noise.

In this <strong><span class="color5">Jupyter Notebook</span></strong>, we will demonstrate how to apply digital filtering techniques to improve the clarity of the signal.

<hr>

## Prep - Install module

In [ ]:
!pip install biosignalsnotebooks

## 1 - Importation of the needed packages

In [ ]:
# biosignalsnotebooks own package for loading and plotting the acquired data
import biosignalsnotebooks as bsnb

# Scientific package
from numpy import array, mean, average, linspace, where
from numpy.random import normal

## 2 - Load of acquired ECG data</p>

In [ ]:
# Load of data
data, header = bsnb.load_signal("ecg_4000_Hz", get_header=True)

print(header)
print(data)

In the following cell, some relevant information is stored inside variables. This relevant information includes the channel number and signal acquisition parameters such as resolution and sampling rate.

For a detailed explanation of how to access this info, the <a href="http://notebooks.pluxbiosignals.com/notebooks/Categories/Load/signal_loading_preparatory_steps_rev.html" target="_blank">"Signal Loading - Working with File Header"</a> Notebook should be consulted.

In [ ]:
ch = "CH1" # Channel
sr = header["sampling rate"] # Sampling rate
resolution = header["resolution"] # Resolution (number of available bits)

## 3 - Generation of signal power spectrum by <i>Fast Fourier Transform</i> (FFT)
With this step is possible to observe the frequency composition of ECG signal.

### 3.1 - Store the desired physiological data (channel 1) int an individual variable

In [ ]:
# Acquired data
signal = data[ch]

### 3.2 - Removal of continuous component from our signal (baseline shift through the subtraction of the average value)
This task ensures more stability of our filtering system.

This step is done to remove the DC offset of the signal. The removal of the constant component is helpful for the frequency analysis. It enhances the frequency components and improves the accuracy of the following Fast Fourier Transformation (FFT).

DC Offset with Signals:

* AC Signal: This is a signal that oscillates or varies around a zero voltage level. For example, an ECG signal is primarily an AC signal because it fluctuates both above and below a zero reference point as the heart beats.

* DC Signal: This is a constant signal that does not change over time. It remains at a fixed voltage level. In the case of an ECG, a DC signal might represent interference or noise that is present at a constant level.

In [ ]:
# Baseline shift.
shifted_signal = array(signal) - mean(signal)

Let's compare the raw signal to the shifted signal

In [ ]:
time = bsnb.generate_time(signal, sr)

bsnb.plot([time, time], [signal, shifted_signal], y_axis_label=["Raw Data", "Baseline Shift"],
          grid_lines=1, grid_columns=2, grid_plot=True)

Notice how the signal is now centered around zero, while the caracteristics of the curve are the same.

### 3.3 - Generation of the power spectrum

In [ ]:
# Power spectrum
freq_axis_1, power_spect_1 = bsnb.plotfft(shifted_signal, sr)

The function *plotfft* utilizes the *fft* function from the numpy module.
Going into the details of the Fast Fourier Transformation is not in the scope of this exercise. Note in the next plot, what happened: The signal was transfered from the time domain to the frequency domain (label of the x-axis). This allows us to analyze which frequencies are present in the signal.

## 4 - The informational content of ECG signal is typically contained below the 40 Hz frequency component
With the next representation we can conclude that there exists some unwanted information out of this frequency band.

In [ ]:
fig_1 = bsnb.plot_informational_band(freq_axis_1, power_spect_1, shifted_signal, sr, band_begin=0.5, band_end=40, legend="ECG Power Spectrum",
                                     x_lim=[0, 100], y_lim=[0, 5e6], show_plot=True)

## 5 - Application of a low-pass filter in order to exclude the unwanted information above the 40 Hz frequency component
Some low-frequency noise can be present at [0, 0.5] Hz frequency band. To exclude it we can follow an identical procedure, but, instead of applying a low-pass filter, a band-pass filter should be used for the frequencies within [0.5, 40] Hz.
<br>
For now, we focus on the more problematic type of noise, i.e., the high frequency noise

In [ ]:
# Digital lowpass filtering with a cutoff frequency f of 40 Hz
filter_signal_1 = bsnb.lowpass(shifted_signal, f=40, order=1, fs=sr)

# Power spectrum
freq_axis_2, power_spect_2 = bsnb.plotfft(filter_signal_1, sr)

The *lowpass* function applies a Butterworth digital filter by using the functions *butter* and *lfilter* from the module scipy.signal. The following two lines are taken from the *lowpass* function:

```python
"""
Parameters
    ----------
    s: array-like
        signal
    f: int
        the cutoff frequency
    order: int
        Butterworth filter order
    fs: float
        sampling frequency
    ----------
"""

  b, a = butter(order, f / (fs/2))
  lfilter(b, a, s)
```
The first line designs a filter of the given order and returns the filter coefficients. The second parameter identifies the critical frequency $W_{n}$. For a Butterworth filter, this is the point at which the gain drops to 1/sqrt(2) that of the passband (the “-3 dB point”). Note that $W_{n}$ is normalized by the Nyquist frequency $f_s/2$

In our case $W_{n} = 40 / (4000/2)= 0,02$

The returned variables $a$ and $b$ are polynomials that are used for the IIR-filter in the next line. An Infinite Impulse Response filter works based on the previous input and output signals:

$y[n]=\frac{1}{a_{0}}\left(\sum_{i=0}^{P}b_{i}x[n-i]-\sum_{j=1}^{Q}a_{j}y[n-j]\right)$

where:

* $P$ is the feedforward filter order
* $b_i$ are the feedforward filter coefficients
* $Q$ is the feedback filter order
* $a_i$ are the feedback filter coefficients
* $x[n]$ is the input signal
* $y[n]$ is the output signal

(https://en.wikipedia.org/wiki/Infinite_impulse_response)


## 6 - Comparison of the power spectrum of original and filtered signal

In [ ]:
bsnb.plot_before_after_filter(shifted_signal, sr, band_begin=0.5, band_end=40, x_lim=[0, 100], y_lim=[0, 5e6], show_plot=True)

In this first filtering attempt we used a first order filter (input argument order=1). It can be seen, in the previous figure, that some unwanted information has been removed, unfortunately no filter has an ideal behavior, so despite we specify a high cutoff frequency of 40 Hz, some information above this threshold is maintained after filtering.

The good news are that components greater than 80 Hz are almost completely removed.

The filter performance can be improved by increasing the filter order, because the higher the filter order is, more quickly the transition between the pass and stop band will be. The transition band will be smaller because of a higher attenuation rate (-20 x <strong>order</strong> dB/decade).

However, the filter order must be chosen with precaution in order to avoid system instability. <strong><span class="color4">Magnitude Bode plots</span></strong> are very useful to check the filter response, as can be seen in the figure below, taking into consideration the following Mathematical formulation.

\begin{equation}
    G = -20\times\log_{10}\Bigg(\sqrt{1 + \bigg(\frac{f}{f_c}\bigg)^{2.n}}\Bigg)
\end{equation}

<table width="100%">
    <tr>
        <td width="20%"></td>
        <td width="30%" style="text-align:left;vertical-align:top">$G$ - Gain factor (negative values reveal an attenuation)</td>
        <td width="30%" style="text-align:left;vertical-align:top">$n$ - Filter order (integer)</td>
        <td width="20%"></td>
    </tr>
    <tr>
        <td></td>
        <td style="text-align:left;vertical-align:top">$f_{c}$ - Cutoff frequency of the filter (40 Hz, for the current implementation)</td>
        <td style="text-align:left;vertical-align:top">$f$ - Independent variable (input frequency to be filtered)</td>
        <td></td>
    </tr>
</table>

In [ ]:
bsnb.plot_low_pass_filter_response()

## 7 - Repetition of the filtering stage but using a higher filter order</p>

In [ ]:
# Digital low-pass filtering with a cutoff frequency f of 40 Hz
filter_signal_2 = bsnb.lowpass(shifted_signal, f=40, order=3, fs=sr)

# Power spectrum
freq_axis_3, power_spect_3 = bsnb.plotfft(filter_signal_2, sr)

In [ ]:
bsnb.plot_before_after_filter(signal, sr, band_begin=0.5, band_end=40, order=3, x_lim=[0, 100], y_lim=[0, 5e6],
                           orientation="vert",show_plot=True)

When the noise level is low, it may be difficult to observe its influence in time domain.
In order to the digital filtering stage produce a visual effect in time domain, the noise level needs to be high.

So we will add some artificial noise do the signal and see the great impact of digital filtering.

<hr>

## 8 - Addition of artificial noise

*Note: The normal() function from numpy.random draws random samples from a normal distribution. In this case a distribution centered around 0 with a standard deviation of 1000 is used and we draw as many samples from it, as there are signal values in our signal.*

In [ ]:
# Noise samples and translation of the baseline
baseline = average(signal)
baseline_shift = 0.50 * baseline
noisy_signal = signal + normal(0, 1000, len(signal)) + baseline_shift

## 9 - Noisy signal representation</p>

In [ ]:
# Plotting of power spectrum
bsnb.plot(linspace(0, len(noisy_signal) - 1, len(noisy_signal)), noisy_signal, x_axis_label='Sample Number',
          y_axis_label='Raw Data', title="Noisy Signal", y_range=(-1e4, 6e4))

*Note: Notice the difference to the previous example signal in the next plot*

In [ ]:
# Plotting of power spectrum
bsnb.plot(linspace(0, len(signal) - 1, len(signal)), signal, x_axis_label='Sample Number',
          y_axis_label='Raw Data', title="Signal", y_range=(-1e4, 6e4))

## 10 - Digital Filtering Stage</p>

In [ ]:
# Digital low-pass filtering with a cutoff frequency f of 40 Hz
noisy_signal_filter = bsnb.lowpass(noisy_signal, f=40, order=3, fs=sr)

## 11 - Comparison of noisy and filtered signal in time domain</p>

In [ ]:
bsnb.plot([linspace(0, len(noisy_signal) - 1, len(noisy_signal))]*2, [noisy_signal, noisy_signal_filter],
          grid_plot=True, grid_lines=1, grid_columns=2, x_axis_label='Sample Number', y_axis_label='Raw Data',
          title=["Noisy Signal", "Filtered Signal"])

As described previously, no filter has an ideal behavior, but, in spite of not ideal the behavior of real filters is predictable.

For example, for the designed 3rd order Butterworth filter with cutoff frequency of 40 Hz, it is expected that after one decade (40 Hz x 10 = 400 Hz) the relative amplitude will be attenuated by -60 dB (assuming a value of 0.1% of the non-filtered signal).

In [ ]:
# Power spectrum (Noisy signal)
freq_axis_noisy, power_spect_noisy = bsnb.plotfft(noisy_signal, sr)

# Power spectrum (Filtered signal)
freq_axis_filter, power_spect_filter = bsnb.plotfft(noisy_signal_filter, sr)

# Relative amplitude 1 decade after the 40 Hz cutoff frequency --> 400 Hz
# Taking into consideration that the search is sequential, so, only the
# first sample that meets the criterium is relevant.
index_decade = where(freq_axis_noisy >= 400)[0][0]
power_decade_noisy = power_spect_noisy[index_decade]
power_decade_filter = power_spect_filter[index_decade]

In [ ]:
!pip install sty

from sty import fg, rs
print(fg(98,195,238) + "\033[1mRelative Amplitude/Power at 400 Hz frequency component [Noisy Signal]: \033[0m" + fg.rs + str(round(power_decade_noisy, 4)))
print(fg(148,193,30) + "\033[1mRelative Amplitude/Power at 400 Hz frequency component [Filtered Signal]: \033[0m" + fg.rs +
      str(round(power_decade_filter, 4)))
print(fg(232,77,14) + "\033[1m\nRatio between filtered and noisy 400 Hz component [Attenuation]: \033[0m" + fg.rs +
      str(round(power_decade_filter/power_decade_noisy, 4)) + " ~ " + str(round(power_decade_filter/power_decade_noisy, 3)) + " = 0.1 %")

Taking into consideration the previous demonstration, we can understand that the designed filter presents the desired behavior !

Unfortunately, <a href="https://en.wikipedia.org/wiki/Noise_(signal_processing)">noise</a> is present in nearly all signal acquisitions, and even physiological data can act as noise if it interferes with the signal under study. For example, electromyographic (EMG) data may be considered noise when capturing electrocardiographic (ECG) signals.

However, as shown earlier, we can address this challenge effectively by using filters—either analog (pre-acquisition) or digital (post-acquisition). This brief tutorial introduces the fundamental principles of digital filters and guides users through the steps to *design* a widely used <a href="https://en.wikipedia.org/wiki/Butterworth_filter">Butterworth</a> filter.